# **Emotion Detection**

## **Introduction**

**Emode** is an emotion detection application that is written in Jupyter notebook using Python and various libraries.

Emotion detection is the process of detecting a person's emotional state. Some of the common ways to detect emotion is by looking into facial features, voice signals etc.

In this project, I'll be implementing three different types of emotion detection and finding a common pattern for a given input. The three types used here are:

- Facial Recognition (using DeepFace)
- Speech Recognition (using Librosa)
- Text Recognition (using Neattext)

## **Installing and Importing Libraries**

In this project, we are going to use the following libraries:



In [ ]:
# Install the necessary libraries.

In [ ]:
# Import the necessary libraries.
import os
import shutil

## **Creating the folders required for the project**

First, let's create the necessary folders so that we can organize and store different sets of files into different folders for processing.

In [ ]:
# Creating a list of all the required additional folders.
folders = [
    "./logs", "./model",
    "./media", "./media/audio", "./media/audio-chunks", "./media/video", "./media/documents"
]

# Function to create folders.
def create_and_clean_folders(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    elif folder in ["./model/", "./media/audio", "./media/audio-chunks", "./media/documents"]:
        shutil.rmtree(folder)
        os.makedirs(folder)
        
# Create the folders.
for folder in folders:
    create_and_clean_folders(folder)